In [1]:
from vivarium_inputs.interface import get_measure
import gbd_mapping as gbd
import pandas as pd, numpy as np, matplotlib.pyplot as plt
import vivarium_gbd_access
from operator import attrgetter
%load_ext autoreload
%autoreload 2

!date

Thu Mar 12 14:23:18 PDT 2020


### 1. pull prevalence (from the VAD cause)

In [2]:
prev_df = pd.DataFrame()
for location in ['India','Nigeria','Ethiopia']:
    data = get_measure(gbd.causes.vitamin_a_deficiency, "prevalence", location).reset_index()
    data['year'] = data['year'].map(attrgetter('left'))
    data['age_start'] = data['age'].map(attrgetter('left'))
    data['age_end'] = data['age'].map(attrgetter('right'))
    #keep under 5 year old and year 2017
    data = data[(data['age_end'] <= 5) & (data['year'] == 2017)]
    prev_df = prev_df.append(data)

/share/code/yaqiw/miniconda/envs/vivarium_conic_lsff/lib/python3.6/site-packages/vivarium_inputs/validation/raw.py:260: UserWarning: Sequela prevalence data for cause vitamin_a_deficiency may not correctly aggregate up to the cause level in all locations. Sequela models may not be consistent with models for this cause.
  warnings.warn(f"{children.capitalize()} {measure} data for cause {entity.name} may not correctly "
/share/code/yaqiw/miniconda/envs/vivarium_conic_lsff/lib/python3.6/site-packages/vivarium_inputs/validation/raw.py:260: UserWarning: Sequela prevalence data for cause vitamin_a_deficiency may not correctly aggregate up to the cause level in all locations. Sequela models may not be consistent with models for this cause.
  warnings.warn(f"{children.capitalize()} {measure} data for cause {entity.name} may not correctly "
/share/code/yaqiw/miniconda/envs/vivarium_conic_lsff/lib/python3.6/site-packages/vivarium_inputs/validation/raw.py:260: UserWarning: Sequela prevalence data

In [3]:
df_prev = prev_df.melt(id_vars = ['location', 'sex', 'age','year','age_start','age_end'], 
                      var_name = 'draw_number', value_name = 'value')
df_prev.groupby('location')['value'].describe(percentiles=[.025, .975])[['mean','2.5%','97.5%']]

,mean,2.5%,97.5%
location,,,
Ethiopia,0.627735,0.273878,0.949212
India,0.569007,0.219264,0.902697
Nigeria,0.226655,0.146157,0.331253


### 2. pull relative risks for measles, diarrhea, LRI from the VAD risk

In [4]:
rr_df = pd.DataFrame()
for location in ['India','Nigeria','Ethiopia']:
    data = get_measure(gbd.risk_factors.vitamin_a_deficiency, "relative_risk", location).reset_index()
    data['year'] = data['year'].map(attrgetter('left'))
    data['age_start'] = data['age'].map(attrgetter('left'))
    data['age_end'] = data['age'].map(attrgetter('right'))
    #keep under 5 year old, year 2017, parameter cat1
    data = data[(data['age_end'] <= 5) & (data['year'] == 2017) & (data['parameter'] == 'cat1')]
    rr_df = rr_df.append(data)

/share/code/yaqiw/miniconda/envs/vivarium_conic_lsff/lib/python3.6/site-packages/vivarium_inputs/validation/raw.py:1384: UserWarning: Relative_risk for risk_factor vitamin_a_deficiency may be outside the normal range.
  warnings.warn(f'{measure.capitalize()} for {entity.kind} {entity.name} may be outside the normal range.')
/share/code/yaqiw/miniconda/envs/vivarium_conic_lsff/lib/python3.6/site-packages/vivarium_inputs/validation/raw.py:1405: UserWarning: Risk_factor vitamin_a_deficiency relative_risk data may violate the following restrictions:  age restriction.
  warnings.warn(f'{entity.kind.capitalize()} {entity.name} {measure} data may violate the '
/share/code/yaqiw/miniconda/envs/vivarium_conic_lsff/lib/python3.6/site-packages/vivarium_inputs/validation/raw.py:1856: UserWarning: Data does not contain all age groups in restriction range.
  warnings.warn('Data does not contain all age groups in restriction range.')
/share/code/yaqiw/miniconda/envs/vivarium_conic_lsff/lib/python3.6/

In [5]:
##RR for under 5 year old
df_rr = rr_df.melt(id_vars = ['location', 'sex', 'age', 'year', 'affected_entity', 'affected_measure',
                                'parameter', 'age_start', 'age_end'], var_name = 'draw_number', value_name = 'value')
df_rr.groupby(['location','affected_entity'])['value'].describe(percentiles=[.025, .975])[['mean','2.5%','97.5%']]

mean  2.5%     97.5%
location affected_entity                                       
Ethiopia diarrheal_diseases            1.721961   1.0  2.593292
         lower_respiratory_infections  1.164763   1.0  1.538674
         measles                       2.257330   1.0  4.473485
India    diarrheal_diseases            1.721961   1.0  2.593292
         lower_respiratory_infections  1.164763   1.0  1.538674
         measles                       2.257330   1.0  4.473485
Nigeria  diarrheal_diseases            1.721961   1.0  2.593292
         lower_respiratory_infections  1.164763   1.0  1.538674
         measles                       2.257330   1.0  4.473485

In [6]:
###Global, all age, all year
df = get_measure(gbd.risk_factors.vitamin_a_deficiency, "relative_risk", "Global").reset_index()
df[df['parameter'] == 'cat1'].melt(id_vars = ['location', 'sex', 'age', 'year', 'affected_entity', 'affected_measure',
                'parameter'], var_name = 'draw_number', value_name = 'value').groupby('affected_entity')['value'].describe(percentiles=[.025, .975])[['mean','2.5%','97.5%']]

/share/code/yaqiw/miniconda/envs/vivarium_conic_lsff/lib/python3.6/site-packages/vivarium_inputs/validation/raw.py:1384: UserWarning: Relative_risk for risk_factor vitamin_a_deficiency may be outside the normal range.
  warnings.warn(f'{measure.capitalize()} for {entity.kind} {entity.name} may be outside the normal range.')
/share/code/yaqiw/miniconda/envs/vivarium_conic_lsff/lib/python3.6/site-packages/vivarium_inputs/validation/raw.py:1405: UserWarning: Risk_factor vitamin_a_deficiency relative_risk data may violate the following restrictions:  age restriction.
  warnings.warn(f'{entity.kind.capitalize()} {entity.name} {measure} data may violate the '
/share/code/yaqiw/miniconda/envs/vivarium_conic_lsff/lib/python3.6/site-packages/vivarium_inputs/validation/raw.py:1856: UserWarning: Data does not contain all age groups in restriction range.
  warnings.warn('Data does not contain all age groups in restriction range.')
/share/code/yaqiw/miniconda/envs/vivarium_conic_lsff/lib/python3.6/

,mean,2.5%,97.5%
affected_entity,,,
diarrheal_diseases,1.125558,1.0,2.496704
lower_respiratory_infections,1.028654,1.0,1.395570
measles,1.218666,1.0,3.809008
